In [5]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

data_dir = '../data/dataDDR'
dataset_dir = '../data/datasetDDR'

##ESCOGER MENOR TAMAÑO DE CARPETA

folders = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]
tamanos =[]
for folder in folders:
    folder_path = os.path.join(data_dir, folder)
    file_count = len(os.listdir(folder_path))
    tamanos.append(file_count)

menor = min(tamanos)
print(menor)

##LIMITAR A MENOR TAMAÑO DE CARPETA
carpeta_fuente = data_dir
carpeta_destino = dataset_dir

carpetas = os.listdir(carpeta_fuente)

for carpeta in carpetas:
    if not carpeta.startswith("."):
        imagenes = os.listdir(carpeta_fuente + '/' + carpeta)

        for i, nombreimg in enumerate(imagenes):
            if i < 42:
                if not nombreimg.startswith("."):
                    shutil.copy(carpeta_fuente + '/' + carpeta + '/' + nombreimg, carpeta_destino + '/' + carpeta + '/' + nombreimg)

##CREAR GENERADORES DE IMAGENES CON DATA AUGMENTATION
datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range = 30,
    width_shift_range = 0.10,
    height_shift_range = 0.10,
    zoom_range = [0.5, 1.5],
    validation_split=0.2 #20% para pruebas
)

##GENERADORES PARA ENTRENAMIENTO Y PRUEBAS
data_gen_entrenamiento = datagen.flow_from_directory(dataset_dir, target_size=(224,224),
                                                     batch_size=32, shuffle=True, subset='training')
data_gen_pruebas = datagen.flow_from_directory(dataset_dir, target_size=(224,224),
                                                     batch_size=32, shuffle=True, subset='validation')

42
Found 237 images belonging to 7 classes.
Found 56 images belonging to 7 classes.


In [2]:
import tensorflow as tf
import tensorflow_hub as hub

url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
mobilenetv2 = hub.KerasLayer(url, input_shape=(224,224,3))

#Congelar el modelo descargado
mobilenetv2.trainable = False

In [3]:
modelo = tf.keras.Sequential([
    mobilenetv2,
    tf.keras.layers.Dense(7, activation='softmax')
])
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 7)                 8967      
                                                                 
Total params: 2266951 (8.65 MB)
Trainable params: 8967 (35.03 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________


In [5]:
from tensorflow.keras.models import load_model

#Compilar como siempre
modelo.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])





SyntaxError: '(' was never closed (3378427090.py, line 5)

In [ ]:
import spicy

#Entrenar el modelo
EPOCAS = 50

historial = modelo.fit(
    data_gen_entrenamiento, epochs=EPOCAS, batch_size=32,
    validation_data=data_gen_pruebas
)

In [ ]:
from tensorflow.keras.models import save_model

save_model(modelo, './models/modeloDDR.h5')

In [ ]:
#Graficas de precisión
acc = historial.history['accuracy']
val_acc = historial.history['val_accuracy']

loss = historial.history['loss']
val_loss = historial.history['val_loss']

rango_epocas = range(50)

plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(rango_epocas, acc, label='Precisión Entrenamiento')
plt.plot(rango_epocas, val_acc, label='Precisión Pruebas')
plt.legend(loc='lower right')
plt.title('Precisión de entrenamiento y pruebas')

plt.subplot(1,2,2)
plt.plot(rango_epocas, loss, label='Pérdida de entrenamiento')
plt.plot(rango_epocas, val_loss, label='Pérdida de pruebas')
plt.legend(loc='upper right')
plt.title('Pérdida de entrenamiento y pruebas')
plt.show()

In [ ]:
from tensorflow.keras.models import load_model

custom_objects = {'KerasLayer': hub.KerasLayer}
modelo = load_model('modelo.h5', custom_objects=custom_objects)